# Minimum Entropy Loss
https://discuss.pytorch.org/t/calculating-the-entropy-loss/14510

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt; plt.style.use('dark_background')
π = np.pi
from numpy.random import uniform as uni
from tqdm import tqdm

In [ ]:
# generate a random signal as a sum of random frequencies
N_FREQS = 5
N_SAMPLES = 100

# generate random frequencies
fs = uni(0, N_SAMPLES/50, N_FREQS)
As = uni(0, 1, N_FREQS)
φs = uni(0, 2*π, N_FREQS)
t = np.arange(N_SAMPLES)

# generate the signal
x = np.sum([As[i]*np.sin(2*π*fs[i]*t+φs[i]) for i in range(N_FREQS)], axis=0)
# x = uni(-uni(.2, 1.2), +uni(.2, 1.2), N_SAMPLES)

#plot the signal
plt.figure(figsize=(10, 2))
plt.scatter(t,x, s=1)
plt.title('Signal')
plt.show()

In [ ]:
# quantize the signal
EPSI = 0.1

nlevels = 1+np.ceil(np.max(np.abs(x))/EPSI)
levels = EPSI*np.arange(-nlevels, nlevels+1)
nlevels = int(1+2*np.ceil(np.max(np.abs(x))/EPSI))
print(f'levels: {levels}')
print(f'number of levels: {nlevels}')

# def quantize(x, ε): return ε*np.round(x/ε) 
def quantize(x, ε): return 2*ε*np.round(x/(2*ε)) 

xq = quantize(x, EPSI)

print(f'levels: {np.unique(xq)}')
print(f'number of levels: {np.unique(xq).shape[0]}')

# plot the quantized signal
plt.figure(figsize=(10, 2))
# plt.plot(t,x, label='original')
plt.scatter(t,xq, label='quantized', s=1)
plt.title('Quantized Signal')
plt.show()

In [ ]:
# measure entropy of the signal
def entropy(x):
    _, counts = np.unique(x, return_counts=True)
    p = counts/len(x)
    # return -np.sum(p*np.log2(p))
    return -np.sum(p*np.log(p))

print(f'Entropy of the quantized signal: {entropy(xq):.2f}')

In [ ]:
class HLoss(nn.Module):
    def __init__(self):
        super(HLoss, self).__init__()

    def forward(self, x):
        assert x.size(-1) == nlevels, f'x.size: {x.size()}'
        b = F.softmax(x, dim=-1) * F.log_softmax(x, dim=-1)
        b = -1.0 * b.sum()
        return b

In [ ]:
# convert the signal to a tensor
xqt = torch.tensor(xq/EPSI + nlevels//2, dtype=torch.int64)
# #convert to a vector of 1-hot encoded values
xq_1hot = F.one_hot(xqt, num_classes=nlevels)
# plot the 2dmatrix 
plt.figure(figsize=(10, 2))
plt.imshow(xq_1hot.T, aspect='auto', interpolation='none', origin='lower')
plt.title('Quantized Signal')
plt.show()

In [ ]:
# calculate entropy
hloss = HLoss()
h = hloss(xq_1hot.to(torch.float32).unsqueeze(0))/N_SAMPLES
print(f'Entropy of the quantized signal: {h:.2f}')

# ENTROPY IS NOT DIFFERENTIABLE
## But apparently these mutherfuckers found a way to do it
$$
 \frac{\partial{H}}{\partial{r_i}} = \lim_{b \to \infty} \sum_{j=0}^{|S|} [1 + \ln p(s_j)] * R(r_i - s_j)
$$

with $R$:

$$
R(r_i - s_j) = \frac{b}{|r|\varepsilon^b} \frac{(r_i - s_j)^{b-1}}{\left[\frac{(r_i -
s_j)^b}{\varepsilon^b} + 1\right]^2} $$

Master thesis version:

$$ 
R = \frac{b}{\left( \text{size}(rq) \cdot \varepsilon^b \right)} \cdot \frac{(rq - s_j)^{b-1}}{\left( \frac{(rq - s_j)^b}{\varepsilon^b} + 1 \right)^2}
$$

In [ ]:
# see what this fucking function actually looks like
def dentropy(rq, b=10.0, ε=0.1):
    symbols, counts = np.unique(rq, return_counts=True)
    p = counts/len(rq)
    # logp = np.log2(p + 1e-8)
    logp = np.log(p + 1e-8)
    H = -np.sum(p*logp) # entropy
    sizer = len(rq)
    DH = 0
    for j in range(len(symbols)):
        DH += (1+logp[j])*b / (sizer*ε**b) * (rq-symbols[j])**(b-1) / (((rq-symbols[j])/ε)**b+1)**2
    return H, DH

H, DH = dentropy(xq, b=10, ε=EPSI)

print(f'Entropy: {H:.2f}')
print(f'Gradient: {DH}')

In [ ]:
def dentropy2(rq, ε=0.1): # importance sampling based entropy calculation #https://en.wikipedia.org/wiki/Kernel_density_estimation
    def normal(x, μ, σ): return np.exp(-0.5*((x-μ)/σ)**2)/(σ*np.sqrt(2*π))
    
    # sample m points from a isotropic gaussian
    m = 300
    samples = np.random.randn(m)
    # samples = np.linspace(-1, 1, m)
    likelihoods = normal(samples, 0, 1)

    σ = 5*ε

    #calculate pdf of the quantized signal
    tot = 0
    for s,l in zip(samples, likelihoods):
        p = np.mean(normal(s, rq, σ))
        ent = -p*np.log(p+1e-8) / l
        tot += ent
    entropy = tot/m 

    return entropy

H = dentropy2(xq, ε=EPSI)
print(f'Entropy: {H:.2f}')

In [ ]:
def entropy_pt(rq, ε=0.1): #https://en.wikipedia.org/wiki/Kernel_density_estimation
    def normal(x, μ, σ): return torch.exp(-0.5*((x-μ)/σ)**2)/(σ*np.sqrt(2*π))

    # rq = rq-torch.mean(rq)
    σ = 5*ε # width of the gaussian kernel
    # sample m points from a isotropic gaussian
    m = 300
    samples = torch.randn(m)
    likelihoods = normal(samples, 0, 1)
    #calculate pdf of the quantized signal
    ent = 0
    for s,l in zip(samples, likelihoods):
        p = torch.mean(normal(s, rq, σ))
        ent += -p*torch.log(p+1e-8) / l
    return ent/m

#create a nn module from the entropy function
class HLoss2(nn.Module):
    def __init__(self, ε=0.1):
        super(HLoss2, self).__init__()
        self.ε = ε
    
    def normal(self, x, μ, σ): return torch.exp(-0.5*((x-μ)/σ)**2)/(σ*np.sqrt(2*π))
    def forward(self, x1, x2):
        r = x1 - x2
        
        σ = 5*self.ε # width of the gaussian kernel
        # sample m points from a isotropic gaussian
        m = 300
        samples = torch.randn(m)
        likelihoods = self.normal(samples, 0, 1)
        #calculate pdf of the quantized signal
        ent = 0
        for s,l in zip(samples, likelihoods):
            p = torch.mean(self.normal(s, rq, σ))
            ent += -p*torch.log(p+1e-8) / l
        return ent/m

## Let's see if there is correlation between the softmax differentiable function and the real entropy

In [ ]:
# test on a lot of tries
H1s, H2s = [], []
for _ in tqdm(range(2000)):
    # generate random frequencies
    fs = uni(0, N_SAMPLES/50, N_FREQS)
    As = uni(0, 1, N_FREQS)
    φs = uni(0, 2*π, N_FREQS)
    t = np.arange(N_SAMPLES)

    # generate the signal
    x = np.sum([As[i]*np.sin(2*π*fs[i]*t+φs[i]) for i in range(N_FREQS)], axis=0)

    # quantize the signal
    xqi = quantize(x, EPSI)

    nlevels = int(1+2*np.ceil(np.max(np.abs(x))/EPSI))

    # measure entropy of the signal
    H1s.append(entropy(xqi)) 

    # # H2
    # # convert the signal to a tensor
    # xqit = torch.tensor(xqi/EPSI + nlevels//2, dtype=torch.int64)
    # #convert to a vector of 1-hot encoded values
    # xqi_1hot = F.one_hot(xqit, num_classes=nlevels)
    # # calculate entropy 2 
    # h = hloss(xqi_1hot.to(torch.float32).unsqueeze(0))/N_SAMPLES
    # H2s.append(h)
    
    xqi_pt = torch.tensor(xqi)
    # xqi_pt = torch.tensor(x)
    H2s.append(entropy_pt(xqi_pt, ε=EPSI).item())

    

H1s, H2s = np.array(H1s), np.array(H2s)

# get the best linear fit between H1s and H2s
A = np.vstack([H1s, np.ones(len(H1s))]).T
m, c = np.linalg.lstsq(A, H2s, rcond=None)[0]
print(f'best fit: y = {m:.2f}x + {c:.2f}')

plt.figure(figsize=(10, 5))
plt.scatter(H1s, H2s, s=5)
plt.plot(H1s, m*H1s + c, color='red')
plt.xlabel('Entropy 1')
plt.ylabel('Entropy 2')
plt.show()


In [ ]:
# criterion = HLoss()
# x = Variable(torch.randn(10, 10))
# w = Variable(torch.randn(10, 3), requires_grad=True)
# output = torch.matmul(x, w)
# loss = criterion(output)
# loss.backward()
# print(w.grad)